<center><h2> Domestic Airfares from ANAC (Agência Nacional de Aviação Civil) </h2></center>
<p><b>Description: </b> This project aims to collect, clean, and transform data from Brazil domestic airfares spanning January 2023 to March 2024 for further analysis.</p> </br></br>


<b>References: </b> <li style="margin-left: 5px"><a href="https://sas.anac.gov.br/sas/downloads/view/frmDownload.aspx?tema=14">Brazil domestic airfares data</a> </br>
<li style="margin-left: 5px"><a href="https://www.anac.gov.br/acesso-a-informacao/dados-abertos/areas-de-atuacao/voos-e-operacoes-aereas/tarifas-aereas-domesticas/46-tarifas-aereas-domesticas">Brazil domestic airfares metadata</a> </br>
<li style="margin-left: 5px" ><a href="https://www.gov.br/anac/pt-br/assuntos/regulados/aerodromos/lista-de-aerodromos-civis-cadastrados">Registered aerodromes data</a>




<h3>Input</h3>
<h4>Input libraries</h4>

In [2]:
import pandas as pd
import glob
import os

from data_processing import Data

<h4>Collect data</h4>
<p>The data collected from the ANAC website are split monthly, so the first thing to do is to join those files into a single CSV.</p>

In [117]:
def join_csv_files(path):
    arquivos = sorted(glob.glob(path+'/*.csv'))

    dfs = []

    for arquivo in arquivos:
        df = pd.read_csv(arquivo, sep=';', usecols=lambda column: column not in ["Unnamed: 0"])
        dfs.append(df)

    df_final = pd.concat(dfs, ignore_index=True)

    df_final_path = path+'.csv'
    if not os.path.exists(df_final_path):
        df_final.to_csv(df_final_path)

join_csv_files('dados/anac_2023')
join_csv_files('dados/anac_2024')

<p>Creating a DataFrame with the CSV data using a class I've built using Pandas.</p>

In [3]:
anac_2023_data = Data('dados/anac_2023.csv')
anac_2023_data.data.head()


,ANO,MES,EMPRESA,ORIGEM,DESTINO,TARIFA,ASSENTOS
0,2023,1,ABJ,SBSV,SIRI,"650,00",17
1,2023,1,ABJ,SBSV,SIRI,"850,00",23
2,2023,1,ABJ,SBSV,SIRI,"1050,00",6
3,2023,1,ABJ,SBSV,SIRI,"1250,00",1
4,2023,1,ABJ,SBSV,SNCL,"450,00",1


In [4]:
anac_2023_data.data.shape[0]

5139219

In [5]:
anac_2024_data = Data('dados/anac_2024.csv')
anac_2024_data.data.head()

,ANO,MES,EMPRESA,ORIGEM,DESTINO,TARIFA,ASSENTOS
0,2024,1,ABJ,SBSV,SDLO,"600,00",8
1,2024,1,ABJ,SBSV,SDLO,"700,00",8
2,2024,1,ABJ,SBSV,SDLO,"900,00",14
3,2024,1,ABJ,SBSV,SDLO,"1550,00",1
4,2024,1,ABJ,SBSV,SIRI,"650,00",11


In [6]:
anac_2024_data.data.shape[0]

1522691

Concatenating the `anac_2023` and `anac_2024` DataFrames.

In [7]:
anac_data = anac_2023_data.concat(anac_2024_data)
anac_data.data.head()

,ANO,MES,EMPRESA,ORIGEM,DESTINO,TARIFA,ASSENTOS
0,2023,1,ABJ,SBSV,SIRI,"650,00",17
1,2023,1,ABJ,SBSV,SIRI,"850,00",23
2,2023,1,ABJ,SBSV,SIRI,"1050,00",6
3,2023,1,ABJ,SBSV,SIRI,"1250,00",1
4,2023,1,ABJ,SBSV,SNCL,"450,00",1


In [8]:
anac_data.data.shape[0]
# 1522691 + 5139219 = 6661910

6661910

<h4>Clean and Trasform data</h4>

Checking column data types.

In [9]:
anac_data.data.dtypes

ANO          int64
MES          int64
EMPRESA     object
ORIGEM      object
DESTINO     object
TARIFA      object
ASSENTOS     int64
dtype: object

The TARIFA column must be a number because it represents the price paid. Therefore, let's change its type.

In [11]:
anac_data.str_to_num('TARIFA')
anac_data.data.dtypes

ANO           int64
MES           int64
EMPRESA      object
ORIGEM       object
DESTINO      object
TARIFA      float64
ASSENTOS      int64
dtype: object

In [12]:
anac_data.data.head()

,ANO,MES,EMPRESA,ORIGEM,DESTINO,TARIFA,ASSENTOS
0,2023,1,ABJ,SBSV,SIRI,650.0,17
1,2023,1,ABJ,SBSV,SIRI,850.0,23
2,2023,1,ABJ,SBSV,SIRI,1050.0,6
3,2023,1,ABJ,SBSV,SIRI,1250.0,1
4,2023,1,ABJ,SBSV,SNCL,450.0,1


Checking for any null values.

In [13]:
anac_data.data.isnull().any()

ANO         False
MES         False
EMPRESA     False
ORIGEM      False
DESTINO     False
TARIFA      False
ASSENTOS    False
dtype: bool

All the column data formats are correct, but the ORIGEM and DESTINO columns are coded. To decode these columns, I retrieved files from the ANAC website that describe the aerodrome names, their cities, and their UFs (states).

In [15]:
anac_aerodromos_publicos = pd.read_csv('dados/cadastro-de-aerodromos-civis-publicos.csv', skiprows=1,delimiter=';')
anac_aerodromos_publicos.head()

,CÓDIGO OACI,CIAD,NOME,MUNICÍPIO ATENDIDO,UF,LATITUDE,LONGITUDE,ALTITUDE,OPERAÇÃO,OACI,...,FREQ. SEMANAL,r1,r2,REFERÊNCIA(S),RESTRIÇÃO(ÕES),SITUAÇÃO,r1.1,REFERÊNCIA,SITUADO NA AMAZÔNIA LEGAL,OBSERVAÇÕES
0,SBAA,PA0008,CONCEIÇÃO DO ARAGUAIA,CONCEIÇÃO DO ARAGUAIA,PA,8° 20' 55'' S,49° 18' 11'' W,199 m,VFR Diurno/Noturno e IFR Diurno,-,...,-,-,-,-,-,Proibição de operações de pouso de aeronaves d...,PA2019-1362,PA2019-1362,X,-
1,SBAE,SP0010,BAURU/AREALVA,BAURU,SP,22° 9' 28'' S,49° 4' 6'' W,598 m,VFR Diurno/Noturno e IFR Diurno/Noturno,-,...,-,-,-,-,-,-,-,-,NaN,-
2,SBAQ,SP0012,BARTOLOMEU DE GUSMÃO,ARARAQUARA,SP,21° 48' 43'' S,48° 7' 59'' W,711 m,VFR Diurno/Noturno e IFR Diurno/Noturno,-,...,-,-,-,-,-,-,-,-,NaN,-
3,SBAR,SE0001,SANTA MARIA,ARACAJU,SE,10° 59' 7'' S,37° 4' 24'' W,7 m,VFR Diurno/Noturno e IFR Diurno/Noturno,RS,...,NaN,PA2022-7303 / PA2020-0406 / PA2021-4220,-,PA2022-7303 / PA2020-0406 / PA2021-4220,-,-,-,-,NaN,-
4,SBAT,MT0003,PILOTO OSVALDO MARQUES DIAS,ALTA FLORESTA,MT,9° 51' 59'' S,56° 6' 18'' W,289 m,VFR Diurno/Noturno e IFR Diurno/Noturno,-,...,-,-,-,-,-,-,-,-,X,-


In [16]:
anac_aerodromos_privados = pd.read_csv('dados/AerodromosPrivados.csv', delimiter=';', encoding='ISO-8859-1', skiprows=1)
anac_aerodromos_privados.head()

,Código OACI,CIAD,Nome,Município,UF,Longitude,Latitude,Altitude,Operação Diurna,Operação Noturna,...,Superfície 1,Designação 2,Comprimento 2,Largura 2,Resistência 2,Superfície 2,Portaria de Registro,Link Portaria,LATGEOPOINT,LONGEOPOINT
0,SI8V,AM0101,FAZENDA MANU,LÁBREA,AM,"065°42'54""W","09°24'14""S",",0",VFR,Sem Operação,...,Cascalho,02/20,"1200,0000","18,00","5700 Kg / 0,50 MPa",Cascalho,PA2022-8592,https://pergamum.anac.gov.br/arquivos/PA2022-8...,"-9,4038889","-65,715"
1,SN33,AM0095,Estância Buriti,AUTAZES,AM,"059°46'38""W","03°32'40""S","31,0",VFR,Sem Operação,...,Terra,14/32,"500,0000","18,00","5700 Kg / 0,50 MPa",Terra,PA2022-9754,https://pergamum.anac.gov.br/arquivos/PA2022-9...,"-3,5444444","-59,777222"
2,SSVC,MS0533,Fazenda Vazante,AQUIDAUANA,MS,"056°09'42""W","20°03'48""S","124,0",VFR / IFR,VFR / IFR,...,Grama,15/33,"2100,0000","45,00",44/F/B/X/T,Asfalto,PA2002-2077,https://pergamum.anac.gov.br/arquivos/PA2002-2...,"-20,063333","-56,161667"
3,SJ4S,PR0191,Fazenda Candoara,CANDÓI,PR,"052°15'32""W","25°40'41""S","723,0",VFR,VFR,...,Cascalho,08/26,"650,0000","16,00","5600 Kg / 1,50 MPa",Cascalho,PA2022-8572,https://pergamum.anac.gov.br/arquivos/PA2022-8...,"-25,678056","-52,258889"
4,SBGP,SP0075,EMBRAER - Unidade Gavião Peixoto,GAVIÃO PEIXOTO,SP,"048°24'17""W","21°45'52""S","609,0",NaN,NaN,...,Asfalto,02L/20,"1800,0000","30,00",30/F/A/Y/T,Terra,PA2021-5749,https://pergamum.anac.gov.br/arquivos/PA2021-5...,"-21,764444","-48,404722"


There are two files with the codes. I collected the data files and put them in DataFrames. Only four columns are necessary: the OACI code, aerodrome name, city and UF. The two files have different names for these columns, so it's necessary to rename them to concatenate the files.

In [17]:
df_aerodromos_privados = anac_aerodromos_privados[['Código OACI', 'Nome','Município','UF']].rename(columns={'Nome': 'NOME AERODROMO','Código OACI':'CÓDIGO OACI','Município':'MUNICÍPIO ATENDIDO'})
df_aerodromos_privados.head()

,CÓDIGO OACI,NOME AERODROMO,MUNICÍPIO ATENDIDO,UF
0,SI8V,FAZENDA MANU,LÁBREA,AM
1,SN33,Estância Buriti,AUTAZES,AM
2,SSVC,Fazenda Vazante,AQUIDAUANA,MS
3,SJ4S,Fazenda Candoara,CANDÓI,PR
4,SBGP,EMBRAER - Unidade Gavião Peixoto,GAVIÃO PEIXOTO,SP


In [18]:
df_aerodromos_publicos = anac_aerodromos_publicos[['CÓDIGO OACI', 'NOME','MUNICÍPIO ATENDIDO','UF']].rename(columns={'NOME': 'NOME AERODROMO'})
df_aerodromos_publicos.head()

,CÓDIGO OACI,NOME AERODROMO,MUNICÍPIO ATENDIDO,UF
0,SBAA,CONCEIÇÃO DO ARAGUAIA,CONCEIÇÃO DO ARAGUAIA,PA
1,SBAE,BAURU/AREALVA,BAURU,SP
2,SBAQ,BARTOLOMEU DE GUSMÃO,ARARAQUARA,SP
3,SBAR,SANTA MARIA,ARACAJU,SE
4,SBAT,PILOTO OSVALDO MARQUES DIAS,ALTA FLORESTA,MT


In [19]:
df_aerodromos = pd.concat([df_aerodromos_publicos, df_aerodromos_privados])
df_aerodromos.head()

,CÓDIGO OACI,NOME AERODROMO,MUNICÍPIO ATENDIDO,UF
0,SBAA,CONCEIÇÃO DO ARAGUAIA,CONCEIÇÃO DO ARAGUAIA,PA
1,SBAE,BAURU/AREALVA,BAURU,SP
2,SBAQ,BARTOLOMEU DE GUSMÃO,ARARAQUARA,SP
3,SBAR,SANTA MARIA,ARACAJU,SE
4,SBAT,PILOTO OSVALDO MARQUES DIAS,ALTA FLORESTA,MT


In [20]:
df_aerodromos.isnull().any()

CÓDIGO OACI            True
NOME AERODROMO        False
MUNICÍPIO ATENDIDO    False
UF                    False
dtype: bool

Some OACI codes are empty, so I will drop those rows.

In [21]:
df_aerodromos.dropna(subset=['CÓDIGO OACI'], inplace=True)

In [22]:
df_aerodromos.isnull().any()

CÓDIGO OACI           False
NOME AERODROMO        False
MUNICÍPIO ATENDIDO    False
UF                    False
dtype: bool

Now I will merge the aerodrome information into the `anac_data` DataFrame based on the OACI code.

In [23]:
anac_data.data['NOME AERODROMO ORIGEM'] = anac_data.data['ORIGEM'].map(df_aerodromos.set_index('CÓDIGO OACI')['NOME AERODROMO'])
anac_data.data['MUNICÍPIO ORIGEM'] = anac_data.data['ORIGEM'].map(df_aerodromos.set_index('CÓDIGO OACI')['MUNICÍPIO ATENDIDO'])
anac_data.data['UF ORIGEM'] = anac_data.data['ORIGEM'].map(df_aerodromos.set_index('CÓDIGO OACI')['UF'])

anac_data.data['NOME AERODROMO DESTINO'] = anac_data.data['DESTINO'].map(df_aerodromos.set_index('CÓDIGO OACI')['NOME AERODROMO'])
anac_data.data['MUNICÍPIO DESTINO'] = anac_data.data['DESTINO'].map(df_aerodromos.set_index('CÓDIGO OACI')['MUNICÍPIO ATENDIDO'])
anac_data.data['UF DESTINO'] = anac_data.data['DESTINO'].map(df_aerodromos.set_index('CÓDIGO OACI')['UF'])

In [24]:
anac_data.data.head()

,ANO,MES,EMPRESA,ORIGEM,DESTINO,TARIFA,ASSENTOS,NOME AERODROMO ORIGEM,MUNICÍPIO ORIGEM,UF ORIGEM,NOME AERODROMO DESTINO,MUNICÍPIO DESTINO,UF DESTINO
0,2023,1,ABJ,SBSV,SIRI,650.0,17,DEPUTADO LUÍS EDUARDO MAGALHÃES,SALVADOR,BA,Barra Grande,MARAÚ,BA
1,2023,1,ABJ,SBSV,SIRI,850.0,23,DEPUTADO LUÍS EDUARDO MAGALHÃES,SALVADOR,BA,Barra Grande,MARAÚ,BA
2,2023,1,ABJ,SBSV,SIRI,1050.0,6,DEPUTADO LUÍS EDUARDO MAGALHÃES,SALVADOR,BA,Barra Grande,MARAÚ,BA
3,2023,1,ABJ,SBSV,SIRI,1250.0,1,DEPUTADO LUÍS EDUARDO MAGALHÃES,SALVADOR,BA,Barra Grande,MARAÚ,BA
4,2023,1,ABJ,SBSV,SNCL,450.0,1,DEPUTADO LUÍS EDUARDO MAGALHÃES,SALVADOR,BA,Lorenzo,CAIRU,BA


In [25]:
anac_data.data.isnull().any()

ANO                       False
MES                       False
EMPRESA                   False
ORIGEM                    False
DESTINO                   False
TARIFA                    False
ASSENTOS                  False
NOME AERODROMO ORIGEM      True
MUNICÍPIO ORIGEM           True
UF ORIGEM                  True
NOME AERODROMO DESTINO     True
MUNICÍPIO DESTINO          True
UF DESTINO                 True
dtype: bool

Some new information is empty. Probably some OACI codes were missing. Let's check.

In [26]:
linhas_origem = anac_data.data.loc[anac_data.data['NOME AERODROMO ORIGEM'].isnull(),'ORIGEM']
linhas_origem.unique()

array(['SBQV', 'SNOB', 'SBFE', 'SBNT'], dtype=object)

In [27]:
linhas_destino = anac_data.data.loc[anac_data.data['NOME AERODROMO DESTINO'].isnull(),'DESTINO']
linhas_destino.unique()

array(['SBQV', 'SNOB', 'SBFE', 'SBNT'], dtype=object)

I conducted a search and retrieved the information for the missing OACI codes. Then, I will merge this information into the `df_aerodromos` DataFrame and fill the missing information in the `anac_data` DataFrame.

In [31]:
codes = ['SBQV', 'SNOB', 'SBFE', 'SBNT']
names = ['Aeroporto Pedro Otacílio Figueiredo','Aeroporto Sobral','João Durval Carneiro','Aeroporto Internacional Augusto Severo']
cities = ['Vitória da Conquista','Sobral','Feira de Santana','Parnamirim']
ufs = ['BA','CE','BA','RN']
data = {'CÓDIGO OACI':codes,'NOME AERODROMO':names,'MUNICÍPIO ATENDIDO':cities,'UF':ufs}
df_oaci_codes = pd.DataFrame(data)
df_oaci_codes.head()

,CÓDIGO OACI,NOME AERODROMO,MUNICÍPIO ATENDIDO,UF
0,SBQV,Aeroporto Pedro Otacílio Figueiredo,Vitória da Conquista,BA
1,SNOB,Aeroporto Sobral,Sobral,CE
2,SBFE,João Durval Carneiro,Feira de Santana,BA
3,SBNT,Aeroporto Internacional Augusto Severo,Parnamirim,RN


In [35]:
df_aerodromos = pd.concat([df_aerodromos, df_oaci_codes])

In [36]:
anac_data.data['NOME AERODROMO ORIGEM'] = anac_data.data['ORIGEM'].map(df_aerodromos.set_index('CÓDIGO OACI')['NOME AERODROMO'])
anac_data.data['MUNICÍPIO ORIGEM'] = anac_data.data['ORIGEM'].map(df_aerodromos.set_index('CÓDIGO OACI')['MUNICÍPIO ATENDIDO'])
anac_data.data['UF ORIGEM'] = anac_data.data['ORIGEM'].map(df_aerodromos.set_index('CÓDIGO OACI')['UF'])

anac_data.data['NOME AERODROMO DESTINO'] = anac_data.data['DESTINO'].map(df_aerodromos.set_index('CÓDIGO OACI')['NOME AERODROMO'])
anac_data.data['MUNICÍPIO DESTINO'] = anac_data.data['DESTINO'].map(df_aerodromos.set_index('CÓDIGO OACI')['MUNICÍPIO ATENDIDO'])
anac_data.data['UF DESTINO'] = anac_data.data['DESTINO'].map(df_aerodromos.set_index('CÓDIGO OACI')['UF'])

In [37]:
anac_data.data.isnull().any()

ANO                       False
MES                       False
EMPRESA                   False
ORIGEM                    False
DESTINO                   False
TARIFA                    False
ASSENTOS                  False
NOME AERODROMO ORIGEM     False
MUNICÍPIO ORIGEM          False
UF ORIGEM                 False
NOME AERODROMO DESTINO    False
MUNICÍPIO DESTINO         False
UF DESTINO                False
dtype: bool

In [42]:
anac_data.data.sample(5)

,ANO,MES,EMPRESA,ORIGEM,DESTINO,TARIFA,ASSENTOS,NOME AERODROMO ORIGEM,MUNICÍPIO ORIGEM,UF ORIGEM,NOME AERODROMO DESTINO,MUNICÍPIO DESTINO,UF DESTINO
4467699,2023,11,AZU,SBUF,SBPA,1290.90,1,PAULO AFONSO,PAULO AFONSO,BA,SALGADO FILHO,PORTO ALEGRE,RS
2250726,2023,6,GLO,SBRF,SBSP,1322.90,2,AEROPORTO INTERNACIONAL RECIFE/GUARARAPES - GI...,RECIFE,PE,SÃO PAULO/CONGONHAS - DEPUTADO FREITAS NOBRE,SÃO PAULO,SP
4299677,2023,11,AZU,SBCB,SBCF,664.90,1,CABO FRIO,CABO FRIO,RJ,TANCREDO NEVES,BELO HORIZONTE,MG
909788,2023,3,AZU,SBVT,SBBE,848.22,2,EURICO DE AGUIAR SALLES,VITÓRIA,ES,INTERNACIONAL DE BELÉM/VAL DE CANS/JÚLIO CEZAR...,BELÉM,PA
4623362,2023,11,TAM,SBGR,SBPJ,639.90,7,GUARULHOS - GOVERNADOR ANDRÉ FRANCO MONTORO,GUARULHOS,SP,BRIGADEIRO LYSIAS RODRIGUES,PALMAS,TO


Saving the final DataFrame to a CSV file.

In [40]:
df_final_path = 'dados/anac_final.csv'
if not os.path.exists(df_final_path):
    anac_data.data.to_csv(df_final_path, index=True)

Now the data is ready for analysis.